In [35]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
import dmba
from dmba import plotDecisionTree, classificationSummary, regressionSummary

%matplotlib inline


In [36]:
df  = pd.read_csv(r'C:\Users\supra\Downloads\ToyotaCorolla.csv')


In [37]:
df.drop(['Model'], axis=1, inplace=True)

In [38]:
data = pd.get_dummies(df, columns=['Fuel_Type'], drop_first=True)


In [39]:
data.drop(['Color'], axis=1, inplace=True)

In [40]:
data.drop(['Id'], axis=1, inplace=True)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Price              1436 non-null   int64
 1   Age_08_04          1436 non-null   int64
 2   Mfg_Month          1436 non-null   int64
 3   Mfg_Year           1436 non-null   int64
 4   KM                 1436 non-null   int64
 5   HP                 1436 non-null   int64
 6   Met_Color          1436 non-null   int64
 7   Automatic          1436 non-null   int64
 8   CC                 1436 non-null   int64
 9   Doors              1436 non-null   int64
 10  Cylinders          1436 non-null   int64
 11  Gears              1436 non-null   int64
 12  Quarterly_Tax      1436 non-null   int64
 13  Weight             1436 non-null   int64
 14  Mfr_Guarantee      1436 non-null   int64
 15  BOVAG_Guarantee    1436 non-null   int64
 16  Guarantee_Period   1436 non-null   int64
 17  ABS           

In [41]:
Y = data.iloc[:, data.columns.get_loc('Price')]

X = data.iloc[:, data.columns != 'Price']


In [43]:
#splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

In [44]:
#feature scaling to be performed on the independent variable 
from sklearn.preprocessing import StandardScaler
sc_X =StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [53]:
# user grid search to find optimized tree
param_grid = {
    'max_depth': [5, 10, 15, 20, 25], 
    'min_impurity_decrease': [0, 0.001, 0.005, 0.01], 
    'min_samples_split': [10, 20, 30, 40, 50], 
}
gridSearch = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, n_jobs=-1)
gridSearch.fit(X_train, y_train)
print('Initial parameters: ', gridSearch.best_params_)

param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
    'min_impurity_decrease': [0, 0.001, 0.002, 0.003, 0.005, 0.006, 0.007, 0.008], 
    'min_samples_split': [14, 15, 16, 18, 20, ], 
}
gridSearch = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, n_jobs=-1)
gridSearch.fit(X_train, y_train)
print('Improved parameters: ', gridSearch.best_params_)

regTree = gridSearch.best_estimator_

Initial parameters:  {'max_depth': 10, 'min_impurity_decrease': 0, 'min_samples_split': 10}
Improved parameters:  {'max_depth': 6, 'min_impurity_decrease': 0.006, 'min_samples_split': 15}


In [54]:
regressionSummary(y_train, regTree.predict(X_train))
regressionSummary(y_test, regTree.predict(X_test))


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 1009.8453
            Mean Absolute Error (MAE) : 735.1250
          Mean Percentage Error (MPE) : -0.9537
Mean Absolute Percentage Error (MAPE) : 7.3119

Regression statistics

                      Mean Error (ME) : -79.1905
       Root Mean Squared Error (RMSE) : 1310.5604
            Mean Absolute Error (MAE) : 934.7660
          Mean Percentage Error (MPE) : -2.0454
Mean Absolute Percentage Error (MAPE) : 9.3875


In [55]:
plotDecisionTree(regTree, feature_names=X_train.columns)
plotDecisionTree(regTree, feature_names=X_train.columns, rotate=True)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'